In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from scrapp_fonction import *
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from dict import months

In [3]:
URL = 'https://www.lequipe.fr/Football/ligue-1/saison-2020-2021/page-calendrier-resultats'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
results = soup.find_all('select', class_='SelectNav__select')

In [4]:
list_url_journey = []
for result in results:
    liens_journey = result.find_all('option')
    for liens in liens_journey:
        list_url_journey.append(liens.get('value'))

In [5]:
row_list = []
for url_journey in list_url_journey:
    journey = pd.read_json(url_journey, lines=True)
    for match in journey['items'][0]:
        try:
            if len(match['items']) == 1:
                match_dict={}
                match_dict['place'] = match['items'][0]['event']['lieu']['ville']
                match_dict['date'] = match['items'][0]['event']['date']
                match_dict['dom_name'] = match['items'][0]['event']['specifics']['domicile']['equipe']['nom']
                match_dict['ext_name'] = match['items'][0]['event']['specifics']['exterieur']['equipe']['nom']
                match_dict['but_dom'] = match['items'][0]['event']['specifics']['score']['domicile']
                match_dict['but_ext'] = match['items'][0]['event']['specifics']['score']['exterieur']
                match_dict['winner'] = match['items'][0]['event']['specifics']['vainqueur']
                match_dict['lien'] = match['items'][0]['event']['lien_web']
                row_list.append(match_dict)
            elif len(match['items']) > 1:
                for m in range(len(match['items'])):
                    match_dict={}
                    match_dict['place'] = match['items'][m]['event']['lieu']['ville']
                    match_dict['date'] = match['items'][m]['event']['date']
                    match_dict['dom_name'] = match['items'][m]['event']['specifics']['domicile']['equipe']['nom']
                    match_dict['ext_name'] = match['items'][m]['event']['specifics']['exterieur']['equipe']['nom']
                    match_dict['but_dom'] = match['items'][m]['event']['specifics']['score']['domicile']
                    match_dict['but_ext'] = match['items'][m]['event']['specifics']['score']['exterieur']
                    match_dict['winner'] = match['items'][m]['event']['specifics']['vainqueur']
                    match_dict['lien'] = match['items'][m]['event']['lien_web']
                    row_list.append(match_dict)
        except:
            pass


In [6]:
data = pd.DataFrame(row_list)

## dataframe Match

### Partie : date

In [7]:
pd.set_option('display.max_rows', None)

In [8]:
df_match = data[['date', 'place']]
df_match.head()

,date,place
0,2020-08-21T19:00:00+0200,Bordeaux
1,2020-08-22T17:00:00+0200,Dijon
2,2020-08-22T21:00:00+0200,Lille
3,2020-08-23T13:00:00+0200,Monaco
4,2020-08-23T15:00:00+0200,Lorient


In [9]:
dates = []
month = []
day = []
year = []

for e in df_match.date:
    date_str = e[0:10]
    date_str = date_str.replace("-", "/")
    year_str = e[0:4]
    month_str = e[5:7]
    day_str = e[8:10]
    year.append(year_str)
    month.append(month_str)
    day.append(day_str)

In [10]:
# df_days = pd.DataFrame(day)
# df_days['day'] = df_days
# dfdays = df_days['day']
# df_match[['day']] = pd.Series([dfdays])
df_match['day']=day

# df_months = pd.DataFrame(month)
# df_months['month'] = df_months
# dfmonths = df_months['month']
# df_match[['month']] = pd.Series([dfmonths])
df_match['month']=month

# df_years = pd.DataFrame(year)
# df_years['year'] = df_years
# dfyears = df_years['year']
# df_match[['year']] = pd.Series([dfyears])
df_match['year']=year

df_match.month = df_match.month.astype(str)

In [11]:
thisdict = {
  "Lille": "nord-pas-de-calais",
  "Paris": "ile-de-france",
  "Monaco": 'provence-alpes-c-te-d-azur',
  "Lyon": "rh-ne-alpes",
  "Marseille": "provence-alpes-c-te-d-azur",
  "Rennes": 'bretagne',
  "Lens": "nord-pas-de-calais",
  "Montpellier": "languedoc-roussillon",
  "Nice": 'provence-alpes-c-te-d-azur',
  "Metz": "lorraine",
  "Saint-Étienne": "rh-ne-alpes",
  "Bordeaux": 'aquitaine',
  "Angers": "pays-de-la-loire",
  "Reims": "champagne-ardenne",
  "Strasbourg": 'alsace',
  "Lorient": "bretagne",
  "Brest": "bretagne",
  "Nantes": "pays-de-la-loire",
  "Nimes": 'languedoc-roussillon',
  "Dijon": 'bourgogne',
}

In [12]:
city = df_match.place
city = city.str.replace('î','i')
mois_chiffre = df_match.month
year = df_match.year
day = df_match.day

### Partie : région

In [13]:
regions = []
list_urls = []

for e in city:
    region = thisdict[e]
    regions.append(region)
    
# df_regions = pd.DataFrame(regions)
# df_regions['region'] = df_regions
# dfregions = df_regions['region']
# df_match[['region']] = pd.Series([dfregions])
df_match['region']=regions
region = df_match.region 
city = city.str.lower()

### Partie : météo - température et pluie

In [14]:
for region, city, year, mois_chiffre, day in zip(df_match['region'], city, year,mois_chiffre, day):
    url = "https://www.historique-meteo.net/france/"+ region + '/' + city + '//' + year + '/' + mois_chiffre + '/' + day + '//'
    list_urls.append(url)

In [15]:
temperature = []
rainfall = []

In [16]:
for url in list_urls:
    try:
        req = requests.get(url)
        soup = BeautifulSoup(req.content, 'html.parser')
        table = soup.find_all('table')
        df = pd.read_html(str(table))
        df = df[0]
        df = df.rename(columns={df.columns[3]: 'Date_day'})
        Précipitations = df[df['Unnamed: 0'] == "Précipitations"]
        Précipitation = Précipitations.Date_day
        Précipitation = str(Précipitation)
        Précipitation = Précipitation[2:8]
        Précipitation = Précipitation.strip()
        if Précipitation == 'ries([':
            Précipitation = None
        Précipitation = int(''.join(i for i in Précipitation if i.isdigit()))
        rainfall.append(Précipitation)
    except Exception:
        rainfall.append(None)
    try:
        req = requests.get(url)
        soup = BeautifulSoup(req.content, 'html.parser')
        table = soup.find_all('table')
        df = pd.read_html(str(table))
        df = df[0]
        df = df.rename(columns={df.columns[3]: 'Date_day'})
        Température = int(df['Date_day'].head(1)[0].split('°')[0])
        temperature.append(Température)
    except Exception:
        temperature.append(None)

In [17]:
# création de la colonne rainfall
# df_rainfall = pd.DataFrame(rainfall)
# df_rainfall['rainfall'] = df_rainfall
# dfrainfall = df_rainfall['rainfall']
# df_match[['rainfall']] = pd.Series([dfrainfall])
df_match['rainfall']=rainfall

# création de la colonne temperature
# df_temperature = pd.DataFrame(temperature)
# df_temperature['temperature'] = df_temperature
# dftemperature = df_temperature['temperature']
# df_match[['temperature']] = pd.Series([dftemperature])
df_match['temperature']=temperature

In [18]:
df_match.rainfall.fillna(df_match.rainfall.mean(), inplace=True)
df_match.temperature.fillna(df_match.temperature.mean(), inplace=True)

In [19]:
df_match.head()

,date,place,day,month,year,region,rainfall,temperature
0,2020-08-21T19:00:00+0200,Bordeaux,21,08,2020,aquitaine,5.0,24.0
1,2020-08-22T17:00:00+0200,Dijon,22,08,2020,bourgogne,19.0,26.0
2,2020-08-22T21:00:00+0200,Lille,22,08,2020,nord-pas-de-calais,1.0,21.0
3,2020-08-23T13:00:00+0200,Monaco,23,08,2020,provence-alpes-c-te-d-azur,1.0,27.0
4,2020-08-23T15:00:00+0200,Lorient,23,08,2020,bretagne,1.0,19.0


In [20]:
#data  = data_match.rename(columns={"TimeStamp": "created_at"})
#data.insert(0, 'updated_at', pd.to_datetime('now').replace(microsecond=0))
#match_table_df = data[['date', 'place', 'rainfall', 'temperature', 'created_at', 'updated_at']]
#match_table_df['date'] = pd.to_datetime(match_table_df['date'])
#match_table_df.to_csv('match_table_final.csv')

In [21]:
df_match['created_at'] = datetime.now()
df_match['updated_at'] = datetime.now()

## Dataframe  goals

### Partie : buteurs, date et ville

In [22]:
list_liens_matchs = data['lien'].tolist()
#list_liens_matchs

In [23]:
buteurs = []
dates = []
match_places = []
match_teams = []

In [24]:
for match in list_liens_matchs:
    req = requests.get(match)
    soup = BeautifulSoup(req.content, 'html.parser')
    goals = soup.find_all('div', class_="Scoreboard__goals")
    
    # on récupère les buteurs des 2 équipes
    for buteur in goals:
        buteur = buteur.text.split()
        buteurs.append(buteur)
        # On récupère l'équipe
        buteur = buteur[-1].strip('’')
        teams = soup.find_all('div', class_='TeamScore__name')
        if buteur[-1].isdigit():
            #domicile
            team = teams[0].text.strip()
            match_teams.append(team)
        else:
            #exterieur
            team = teams[1].text.strip()
            match_teams.append(team)
        # on récupère les lieus ou le match se jouent   
        match_place = soup.find('div', class_="TeamScore__identity")
        match_place = match_place.text.strip()
        match_places.append(match_place)    
        # on récupère la date du match
        date = soup.find('div', class_="Scoreboard__titles pageHeading")
        date = date.text.split()
        if date[-1].isdigit(): 
            date = "-".join(date[-3:])
            dates.append(date)
        else:
            date = soup.find('div', class_="pageHeading pageHeading__subtitle")
            date = date.text.split()
            date = "-".join(date[1:4])
            dates.append(date)

In [25]:
df_buts = pd.DataFrame()
#df_buts[['buteurs', 'dates', 'match_places','teams']] = pd.Series([buteurs, dates, match_places,match_teams])
df_buts['buteurs']=buteurs
df_buts['dates']=dates
df_buts['match_place']=match_places
df_buts['teams']=match_teams

In [26]:
df_buts.head()

,buteurs,dates,match_place,teams
0,"[22’, I., Traoré]",22-Août-2020,Dijon,Angers
1,"[J., Bamba, 40’]",22-Août-2020,Lille,Lille
2,"[74’, D., Da, Silva]",22-Août-2020,Lille,Rennes
3,"[A., Disasi, 45’, +2]",23-Août-2020,Monaco,Monaco
4,"[B., Badiashile, 55’]",23-Août-2020,Monaco,Monaco


### Partie : nom des joueurs

In [27]:
import re
names = []
minutes = []
initial = []
carac = []
goal_type = []

def find_letter(text):
    txt = re.findall(r'[A-Z\]+',text)
    return " ".join(txt)

In [28]:
for minute in df_buts.buteurs:
    minute = ' '.join(minute)
    minute = int(''.join(filter(str.isdigit, minute)))
    minute = str(minute)
    minutes.append(minute)
    
# df_minutes = pd.DataFrame(minutes)
# df_minutes['minute'] = df_minutes
# dfminute = df_minutes['minute']
# df_buts[['minute']] = pd.Series([dfminute])
df_buts['minute']=minutes

for e in df_buts.buteurs:
    penalty = [s for s in e if "(s.p.)" in s]
    own_goal = [s for s in e if "(csc)" in s]
    if len(penalty)>0:
        goal_type.append('penalty')
    elif len(own_goal)>0:
        goal_type.append('own_goal')
    else:
        goal_type.append('normal')
    

df_buts['goal_type']=goal_type

In [29]:
for e in df_buts.buteurs:
    e_clean = [c for c in e if c not in ['(s.p.)', '(csc)']]
    e_clean = [c for c in e_clean if any(chr.isalpha() for chr in c)]
    name = ' '.join(e_clean)
    names.append(name)

In [30]:
# df_names = pd.DataFrame(names)
# df_names['full_name'] = df_names
# dfname = df_names['full_name']
# df_buts[['full_name']] = pd.Series([dfname])
df_buts['full_name']=names

In [31]:
df_buts.head(20)

,buteurs,dates,match_place,teams,minute,goal_type,full_name
0,"[22’, I., Traoré]",22-Août-2020,Dijon,Angers,22,normal,I. Traoré
1,"[J., Bamba, 40’]",22-Août-2020,Lille,Lille,40,normal,J. Bamba
2,"[74’, D., Da, Silva]",22-Août-2020,Lille,Rennes,74,normal,D. Da Silva
3,"[A., Disasi, 45’, +2]",23-Août-2020,Monaco,Monaco,452,normal,A. Disasi
4,"[B., Badiashile, 55’]",23-Août-2020,Monaco,Monaco,55,normal,B. Badiashile
5,"[05’, B., Dia]",23-Août-2020,Monaco,Reims,5,normal,B. Dia
6,"[21’, E., Touré]",23-Août-2020,Monaco,Reims,21,normal,E. Touré
7,"[Y., Wissa, 51’]",23-Août-2020,Lorient,Lorient,51,normal,Y. Wissa
8,"[(s.p.), A., Grbic, 60’]",23-Août-2020,Lorient,Lorient,60,penalty,A. Grbic
9,"[P., Hamel, 87’]",23-Août-2020,Lorient,Lorient,87,normal,P. Hamel


In [32]:
df_buts['goal_type'].unique()

array(['normal', 'penalty', 'own_goal'], dtype=object)

In [33]:
month = []

for e in df_buts.dates:
    e = e.split('-')
    month.append(e[1])

In [34]:
# création de la colonne months
# df_month = pd.DataFrame(month)
# df_month['month'] = df_month
# dfmonth = df_month['month']
# df_buts[['month']] = pd.Series([dfmonth])
df_buts['months']=month

In [35]:
list_month = {
'Janvier': '01',
'Février': '02',
'Mars': '03',
'Avril': '04',
'Mai': '05',
'Juin': '06',
'Juillet':'07',
'Août': '08',
'Septembre': '09',
'Octobre': '10',
'Novembre': '11',
'Décembre': '12'}

In [36]:
for pat, repl in list_month.items():
    df_buts.dates = df_buts.dates.str.replace(pat, repl)

In [37]:
date = []
for e in df_buts.dates:
    e = e.strip(',')
    e = e.split('-')
    e.reverse()
    e = '-'.join(e)
    date.append(e)

In [38]:
df_buts.dates = date

In [39]:
df_buts.drop(['months'], axis=1, inplace=True)

In [40]:
df_buts.head()

,buteurs,dates,match_place,teams,minute,goal_type,full_name
0,"[22’, I., Traoré]",2020-08-22,Dijon,Angers,22,normal,I. Traoré
1,"[J., Bamba, 40’]",2020-08-22,Lille,Lille,40,normal,J. Bamba
2,"[74’, D., Da, Silva]",2020-08-22,Lille,Rennes,74,normal,D. Da Silva
3,"[A., Disasi, 45’, +2]",2020-08-23,Monaco,Monaco,452,normal,A. Disasi
4,"[B., Badiashile, 55’]",2020-08-23,Monaco,Monaco,55,normal,B. Badiashile


In [41]:
df_buts['teams'] = df_buts['teams'].replace(
    {'Paris-SG': 'Paris'})
df_buts['teams'].unique()


array(['Angers', 'Lille', 'Rennes', 'Monaco', 'Reims', 'Lorient',
       'Strasbourg', 'Nîmes', 'Nice', 'Lens', 'Montpellier', 'Lyon',
       'Paris', 'Saint-Étienne', 'Dijon', 'Nantes', 'Bordeaux', 'Brest',
       'Marseille', 'Metz'], dtype=object)

In [42]:
df_buts['match_place'] = df_buts['match_place'].replace(
    {'Paris-SG': 'Paris'})
df_buts['match_place'].unique()


array(['Dijon', 'Lille', 'Monaco', 'Lorient', 'Nîmes', 'Nice',
       'Montpellier', 'Paris', 'Marseille', 'Lyon', 'Rennes',
       'Strasbourg', 'Reims', 'Nantes', 'Saint-Étienne', 'Angers', 'Metz',
       'Brest', 'Lens', 'Bordeaux'], dtype=object)

In [43]:
df_goals = pd.DataFrame()
df_goals['match_id'] = df_buts['dates'] + ' ' + df_buts['match_place']
df_goals['player_id'] = df_buts['full_name'] + ' ' + df_buts['teams']
df_goals['goal_type'] = df_buts['goal_type']
df_goals['created_at'] = datetime.now()
df_goals['updated_at'] = datetime.now()

In [44]:
df_goals.head()

,match_id,player_id,goal_type,created_at,updated_at
0,2020-08-22 Dijon,I. Traoré Angers,normal,2021-07-02 20:04:36.099259,2021-07-02 20:04:36.101412
1,2020-08-22 Lille,J. Bamba Lille,normal,2021-07-02 20:04:36.099259,2021-07-02 20:04:36.101412
2,2020-08-22 Lille,D. Da Silva Rennes,normal,2021-07-02 20:04:36.099259,2021-07-02 20:04:36.101412
3,2020-08-23 Monaco,A. Disasi Monaco,normal,2021-07-02 20:04:36.099259,2021-07-02 20:04:36.101412
4,2020-08-23 Monaco,B. Badiashile Monaco,normal,2021-07-02 20:04:36.099259,2021-07-02 20:04:36.101412


In [45]:
df_goals.to_csv('goals_table_final.csv')